# Puzzler Express, 9 Nov 2018

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import math

Let's set a variable for the total number of numbers available, and build a way to compare them.

In [2]:
tot=10**7
tot

10000000

In [3]:
def sortphone(n):
    return ''.join(sorted("%07d" % n))

def cmp(a, b):
    sa=sortphone(a)
    sb=sortphone(b)
    return(sa == sb)

print(123, 131, cmp(123, 131))
print(123, 132, cmp(123, 132))

123 131 False
123 132 True


Let's build an array of all numbers from "000-0000" to "999-9999", and sort them by digit so that we can use array methods to count the number of permutations of a given number.

In [5]:
SX=np.fromiter((sortphone(x) for x in np.arange(tot)), np.dtype('U7'), count=tot)
SX

array(['0000000', '0000001', '0000002', ..., '7999999', '8999999',
       '9999999'], dtype='<U7')

In [6]:
def count(a):
    sa=sortphone(a)
    return len(SX[SX==sa])

We can use this to count the number of duplicates of a number. If all seven digits of your number different, there are a lot of permutations, and the probability that others have your number is the highest (subtracting 1 from both  the numerator and denominator since your number is removed from the list), approximately ~0.0005039.

In [7]:
(count(1234567)-1)/(tot-1)    # subtract one since I have this one!

0.000503900050390005

Conversely, if all seven digits of your number are identical, *no one* has a permutation of your number, and your probability is 0:

In [8]:
(count(2222222)-1)/(tot-1)

0.0

In general, for a given number I can count (a) how many digits appear more than one, and (b) count the number of time each digit appears more than once. I define $d = \{d_1, d_2, ... d_m\}$, where $d_1$ is the number of times the first replicated digit appears, $d_2$ is the number of times the second replicated digit appears, and so on. 

For example, "445-5566" would produce $d = \{2, 3, 2\}$. If there are no replicated digits, as for "222-2222", $d$ is the empty set.

For a given number with a duplicate pattern of $d$, the number of permutations is given by.

$$ n_{P}(d) = \frac{ {}^{7}\!P_{7} }{ \prod^{m}_{i=1}{d_i!} } = \frac{ 7! }{ \prod^{m}_{i=1}{d_i!} } $$

When all digits are identical, $n_P(\{7\}) = 1$. When all digits are identical, $n_P(\{ \}) = 5040$. Note that the order of the values of $d$ do not affect the value of $n_P$, so we can sort the values of $d$ and get the same results.

We can count the number of replicated digits for all numbers from 000-0000 to 999-9999, and group them by (sorted) $d$. This is kind of the brute-force method, so forgive the inelegance.

In [9]:
memo={}
def count_dups_in_num(s):
    """Determine the (sorted) pattern of replicated numbers in a given number.
        For example, "4455566" would produce (2,2,3), because there are 2
        digits that appear twice ('4' and '6'), and 1 digit that appears
        thrice ('5'). Memoized for performance.
    """
    if s in memo:
        return memo[s]
    d=dict()
    for c in s:
        if c not in d:
            d[c]=0
        d[c]+=1
    t=list()
    for k, v in d.items():
        if v > 1:
            t.append(v)
    ret=tuple(sorted(t))
    memo[s]=ret
    return ret

count_dups_in_num("4455566")

(2, 2, 3)

In [10]:
def count_all_dup_patterns(sx):
    """Apply count_dups_in_num() to every entry"""
    d={"pattern": [], "count": []}
    count={}
    for s in SX:
        dups=count_dups_in_num(s)
        if dups not in count:
            count[dups]=0
        count[dups]+=1
    for k, v in count.items():
        d["pattern"].append(k)
        d["count"].append(v)
    return pd.DataFrame(d)

df=count_all_dup_patterns(SX)

So only $10$ out of $10^7$ numbers have $7$ identical digits (e.g. "111-1111", "222-2222"). Conversely, $604800$ numbers out of $10^7$ have $7$ unique digits.

In [11]:
df

,pattern,count
0,"(7,)",10
1,"(6,)",630
2,"(2, 5)",1890
3,"(5,)",15120
4,"(3, 4)",3150
5,"(2, 4)",75600
6,"(4,)",176400
7,"(3, 3)",50400
8,"(2, 2, 3)",75600
9,"(2, 3)",1058400


For each duplicate pattern $d$, we can calculate the number of permutations ($n_P$) for each number in that set:

In [12]:
def permute_pattern(pat, n):
    num=math.factorial(n)
    den=1
    for p in pat:
        den*=math.factorial(p)
    return num//den
v_permute_pattern=np.vectorize(permute_pattern)

df["dups_of_number"]=v_permute_pattern(df["pattern"], 7)

In [13]:
df["prob_of_dup"]=(df["dups_of_number"]-1)/(tot-1)

In [14]:
df

,pattern,count,dups_of_number,prob_of_dup
0,"(7,)",10,1,0.000000e+00
1,"(6,)",630,7,6.000001e-07
2,"(2, 5)",1890,21,2.000000e-06
3,"(5,)",15120,42,4.100000e-06
4,"(3, 4)",3150,35,3.400000e-06
5,"(2, 4)",75600,105,1.040000e-05
6,"(4,)",176400,210,2.090000e-05
7,"(3, 3)",50400,140,1.390000e-05
8,"(2, 2, 3)",75600,210,2.090000e-05
9,"(2, 3)",1058400,420,4.190000e-05


From here it's pretty easy to calculate the average of the probability of duplicate numbers weighted by the probability (or frequency) that your number contains a particular number of duplicates. 

In [15]:
sum(df["prob_of_dup"]*df["count"])/tot

0.0001677613307761331